In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
class node(object):
    
    def __init__(self,state,inbound=None,outbound=None):
        self.state=state
        self.inbound={}
        self.outbound={}
        
    def __str__(self):
        string="State: "+str(self.state)
        return string
    
    def addInbound(self,comingFrom,arcCost):
        self.inbound[comingFrom]=arcCost
        
    def addOutbound(self,goingTo,arcCost):
        self.outbound[goingTo]=arcCost


In [3]:
class layer(object):
    
    def __init__(self, layer_no):
        self.layer_no=layer_no
        self.nodes={}
        
    def __str__(self):
        string="Layer no. {}, containing {} nodes".format(self.layer_no,len(self.nodes))
        return string
    
    def addNode(self,node):
        self.nodes[node.state]=node
        
    def getNodes(self):
        return self.nodes

In [4]:
class arc(object):
    
    def __init__(self, head, tail, arctype, length):
        self.head=head
        self.tail=tail
        self.arctype=arctype
        self.length=length
    
    def __str__(self):
        string="Arc type: {}, Length: {}, Headnode: {}, Tailnode: {}".format(self.arctype,self.length,self.head.state, self.tail.state)
        return string

In [ ]:
n=6
U=10

np.random.seed(78440)
    
cvals=np.random.randint(1,U,n)
wvals=np.random.randint(1,U,n)
W=sum(wvals)/2
d=[0,1]

In [ ]:
n1=25
U1=50
    
cvals1=np.random.randint(1,U1,n1)
wvals1=np.random.randint(1,U1,n1)
W1=sum(wvals1)/2
d1=[0,1]

def read_instance(address = 'instance.csv'):
    data = pd.read_csv(address)
    c = data['c'].values
    w = data['w'].values
    W = data['W'].values[0]
    return c,w,W

In [ ]:
ntest=4

cvalstest=[7,8,6,14]
wvalstest=[3,3,4,6]
Wtest=6

In [ ]:
def Algorithm_1(n,cvals,wvals,W,d):
    
    root_node=node(0)
    all_layers={}
    all_layers[0]={}
    all_layers[0][root_node.state]=root_node
    
    #Iterate through each layer
    for x in range(1,n):
        #Create next layer
        currLayer=all_layers[x-1]
        all_layers[x]={}
        nextLayer=all_layers[x]
        #Iterate through each node and action decision
        for currW, nodeInstance in currLayer.items():
            for decision in d:
                newstate=currW+(wvals[x-1]*decision)
                #Feasability check
                #Check if the new state already exists in the layer
                if newstate<=W:
                    if newstate in nextLayer:
                        #Add arc into arclist
                        cost=cvals[x-1]*decision
                        #Add arc into node inbound and outbound
                        nodeInstance.outbound[nextLayer[newstate]]=cost
                        nextLayer[newstate].inbound[nodeInstance]=cost
                    else:
                        #Create new node, add arc to arclist
                        new_node=node(newstate)
                        nextLayer[newstate]=new_node
                        cost=cvals[x-1]*decision
                        #Add arc to node inbound and outbound
                        nodeInstance.outbound[new_node]=cost
                        new_node.inbound[nodeInstance]=cost
        
    #Changed from W because Rachel said so
    terminal_node=node(-999)
    all_layers[n]={}
    all_layers[n][-999]=terminal_node
    
    for currW, nodeInstance in all_layers[n-1].items():
        for decision in d:
            newstate=currW+(wvals[n-1]*decision)
            if newstate<=W:
                cost=cvals[n-1]*decision
                nodeInstance.outbound[terminal_node]=cost
                terminal_node.inbound[nodeInstance]=cost
                
    #final_node=node(0)            
    #terminal_node.outbound[final_node]=0
                
    return all_layers

In [ ]:
def Algorithm_2(all_layers,n):
    #Final reduction algorithm here we go
    reduced_layers=all_layers.copy()
    
    #Starting from the third last layer moving up
    for x in range(n-1,0,-1):
        #Start iterating through one path nodes
        for node_1 in list(reduced_layers[x].values()):
            to_remove=[]
            for node_2 in list(reduced_layers[x].values()):
                if(node_1 is not node_2 and node_1 in reduced_layers[x].values() and node_2 in reduced_layers[x].values()):
                    if(node_1.outbound.items()==node_2.outbound.items()):
                        for comingFrom, arcCost in node_2.inbound.items():
                            comingFrom.outbound[node_1]=arcCost
                            node_1.inbound[comingFrom]=arcCost
                            if node_2 in comingFrom.outbound:
                                del(comingFrom.outbound[node_2])
                        to_remove.append(node_2)
            for remove_node in to_remove:
                if remove_node.state in reduced_layers[x]:
                    del(reduced_layers[x][remove_node.state])
                                  
    return reduced_layers

In [ ]:
def Algorithm_2(all_layers,n):
    #Final reduction algorithm here we go
    reduced_layers=all_layers.copy()
    
    #Starting from the third last layer moving up
    for x in range(n-1,0,-1):
        #Start iterating through one path nodes
        for node_1 in list(reduced_layers[x].values()):
            if node_1 in reduced_layers[x].values():
                to_remove=[]
                for node_2 in list(reduced_layers[x].values()):
                    if node_2 in reduced_layers[x].values():
                        if(node_1 is not node_2):
                            if(node_1.outbound.items()==node_2.outbound.items()):
                                for comingFrom, arcCost in node_2.inbound.items():
                                    comingFrom.outbound[node_1]=arcCost
                                    node_1.inbound[comingFrom]=arcCost
                                    if node_2 in comingFrom.outbound:
                                        del(comingFrom.outbound[node_2])
                                to_remove.append(node_2)
                            
            for remove_node in to_remove:
                if remove_node.state in reduced_layers[x]:
                    del(reduced_layers[x][remove_node.state])
            
                                  
    return reduced_layers

In [ ]:
[cvals2,wvals2,W2]=read_instance()
n2=len(cvals2)

In [ ]:
start1=time.time()
all_layers=Algorithm_1(n2,cvals2,wvals2,W2,d)
end1=time.time()
reduced_layers=Algorithm_2(all_layers,n2)
end2=time.time()

In [ ]:
#Just printing stuff, no biggie
    
print("Algo 1 Runtime: {}ms".format(1000*(end1-start1)))
print("Algo 2 Runtime: {}ms".format(1000*(end2-end1)))
print("Total Runtime: {}ms".format(1000*(end2-start1)))

In [ ]:
#Printing node and layer information
for key, layer in all_layers.items():
    print("Layer no: {}".format(key))
    print("Nodes in layer: {}".format(len(layer.values())))
    print("")
    '''for currWeight, nodeInstance in layer.items():
        print("Node: {}".format(currWeight))
        for goingTo, arcCost in nodeInstance.outbound.items():
            print("Arc cost: {}".format(arcCost))
    print("")'''